In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
sc = SparkContext.getOrCreate(conf=conf)
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)
ssc.checkpoint("checkpoint")

In [3]:
socket_stream = ssc.socketTextStream("localhost", 5555)

In [4]:
lines = socket_stream.window(20)

In [5]:
counts = lines.flatMap(lambda text: text.split( " " ))\
    .filter(lambda word: word.lower().startswith("#"))\
    .map(lambda word: (word.lower(), 1))\
    .reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 60*5, 10)

swappedCounts = counts.map(lambda x: (x[1], x[0]))
sortedCounts = swappedCounts.transform(lambda rdd: rdd.sortByKey())

sortedCounts.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2018-02-18 22:38:00
-------------------------------------------
(1, '#rickgates')
(1, '#manafort')
(1, '#trumprussia')
(1, '#privatization')
(1, '#flag')
(1, '#resist')
(1, '#stpatricks')
(1, '#ireland')
(1, '#clover')
(1, '#symbol')
...

-------------------------------------------
Time: 2018-02-18 22:38:10
-------------------------------------------
(1, '#irandeal')
(1, '#c…rt')
(1, '#russianbot')
(1, '#fbi')
(1, '#housekeeper')
(1, '#presidentsday,')
(1, '#abc730')
(1, '#liar')
(1, '#trumptaxscam')
(1, '#thegreatwall')
...



In [ ]:
ssc.stop()

In [ ]:
def updateFunc(new_values, last_sum):
        return sum(new_values) + (last_sum or 0)

In [ ]:
# get the hashtags:
hashtags = lines.flatMap(lambda text: text.split(" "))\
    .filter(lambda word: word.lower().startswith("#"))
    
hashtags.pprint()

In [ ]:
# get the last 5min data reduced every second
counts = lines.flatMap(lambda text: text.split( " " ))\
    .filter(lambda word: word.lower().startswith("#"))\
    .map(lambda word: (word.lower(), 1))\
    .reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 60*5, 1)

counts.pprint()